In [1]:
import numpy as np
import pandas as pd
import os
import scipy.io
from scipy.signal import butter, filtfilt, iirnotch, cheby2
from einops import rearrange
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from lightning import Fabric
import yaml
import lightning
from pytorch_lightning.utilities.model_summary import ModelSummary
from einops import repeat
from typing import Tuple
from torch.nn import functional as F

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from mne.decoding import CSP
from sklearn.metrics import accuracy_score

import sys

sys.path.append("../../")

from classification.loaders import EEGDataset,load_data
from models.unet.eeg_unets import Unet,UnetConfig, BottleNeckClassifier, Unet1D
from classification.classifiers import DeepClassifier
from classification.loaders import subject_dataset, CSP_subject_dataset
from classification.open_bci_loaders import OpenBCIDataset,OpenBCISubject,load_files,CSPOpenBCISubject
from models.unet import base_eegnet
from ntd.networks import LongConv
from ntd.diffusion_model import Diffusion
from ntd.utils.kernels_and_diffusion_utils import WhiteNoiseProcess
from classification_heads import ClassificationHead, EEGNetHead, DiffusionClf
import json

c:\Users\rahar\anaconda3\envs\neuro\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
torch.set_float32_matmul_precision("medium")
FABRIC = Fabric(accelerator="cuda",precision="bf16-mixed")
lightning.seed_everything(0)

Using bfloat16 Automatic Mixed Precision (AMP)
Seed set to 0


0

## Datasets

### Constants

In [3]:
c = np.split(np.arange(3*9),3)
c = np.concatenate([c[0],c[2]])


In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SIGNAL_LENGTH = 512
N_CSP_CHANNELS = 18
N_EEG_CHANNELS = 2

CONF_PATH = "../diffusion/conf"

FS_BCI_COMP = 250
DATA_PATH_BCI_COMP = "../../data/2b_iv"
SAVE_PATH_BCI_COMP = "../../saved_models/raw_eeg"

REAL_DATA_BCI_COMP = "../../data/2b_iv/raw"
CSP_DATA_BCI_COMP = "../../data/2b_iv/csp"

TRAIN_SPLIT_BCI_COMP = 9*[["train"]]
TEST_SPLIT_BCI_COMP = 9*[["test"]]

CHANNELS_BCI_COMP = [0,2]

c = np.split(np.arange(3*9),3)
c = np.concatenate([c[0],c[2]])

CSP_CHANNELS_BCI_COMP = c


train_split = 2*[["train"]]
test_split = 2*[["test"]]
save_path = os.path.join("processed","raw")
small_stride = os.path.join("processed","data/collected_data/small_stride")


In [5]:
TRAIN_SPLIT_OPENBCI = 2*[["train"]]
TEST_SPLIT_OPENBCI = 2*[["test"]]

SAVE_PATH_OPENBCI = os.path.join("processed","raw")
CSP_SAVE_PATH_OPENBCI = os.path.join("processed","data/collected_data/csp")

CSP_CHANNELS_OPENBCI = np.arange(2*9)
CHANNELS_OPENBCI = np.arange(0,2)

In [6]:
with open(os.path.join(CONF_PATH, "train.yaml"), "r") as f:
    train_yaml = yaml.safe_load(f)
    
with open(os.path.join(CONF_PATH, "classifier.yaml"), "r") as f:
    classifier_yaml = yaml.safe_load(f)
    
with open(os.path.join(CONF_PATH, "network.yaml"), "r") as f:
    network_yaml = yaml.safe_load(f)
    
with open(os.path.join(CONF_PATH, "diffusion.yaml"), "r") as f:
    diffusion_yaml = yaml.safe_load(f)

### BCI Competition

#### Normal

In [7]:
dataset = {}
for i in range(1,10):
    mat_train,mat_test = load_data("../../data/2b_iv",i)
    dataset[f"subject_{i}"] = {"train":mat_train,"test":mat_test}

train_dataset_bci_comp = EEGDataset(subject_splits=TRAIN_SPLIT_BCI_COMP,
                    dataset=None,
                    save_paths=[REAL_DATA_BCI_COMP],
                    subject_dataset_type=subject_dataset,
                    channels=CHANNELS_BCI_COMP,
                    sanity_check=False,
                    length=2.05)

test_dataset_bci_comp = EEGDataset(subject_splits=TEST_SPLIT_BCI_COMP,
                    dataset=None,
                    save_paths=[REAL_DATA_BCI_COMP],
                    channels=CHANNELS_BCI_COMP,
                    sanity_check=False,
                    length=2.05)

(3026, 2, 512)
(3026,)
final data shape: (3026, 2, 512)
(2241, 2, 512)
(2241,)
final data shape: (2241, 2, 512)


#### CSP

In [8]:
train_csp_dataset_bci_comp = EEGDataset(subject_splits=TRAIN_SPLIT_BCI_COMP,
                    dataset=None,
                    save_paths=[CSP_DATA_BCI_COMP],
                    subject_dataset_type=CSP_subject_dataset,
                    channels=CSP_CHANNELS_BCI_COMP,
                    sanity_check=False,
                    length=2.05)

test_csp_dataset_bci_comp = EEGDataset(subject_splits=TEST_SPLIT_BCI_COMP,
                    dataset=None,
                    save_paths=[CSP_DATA_BCI_COMP],
                    channels=CSP_CHANNELS_BCI_COMP,
                    sanity_check=False,
                    length=2.05)

(3026, 18, 512)
(3026,)
final data shape: (3026, 18, 512)
(2241, 18, 512)
(2241,)
final data shape: (2241, 18, 512)


### OpenBCI

#### Normal

In [9]:
print(f"path {os.getcwd()}")
files = load_files("../../data/collected_data/")

train_dataset_openbci = OpenBCIDataset(
	subject_splits=TRAIN_SPLIT_OPENBCI,
	dataset=files,
	save_paths=[SAVE_PATH_OPENBCI],
	fake_data=None,
	dataset_type=OpenBCISubject,
	channels=np.arange(0,2),
	subject_channels=["ch2","ch5"],
	stride=128,
	epoch_length=512
)

test_dataset_openbci = OpenBCIDataset(
	subject_splits=TEST_SPLIT_OPENBCI,
	save_paths=[SAVE_PATH_OPENBCI],
	fake_data=None,
	dataset_type=OpenBCISubject,
	channels=np.arange(0,2),
	subject_channels=["ch2","ch5"],
	stride=128,
	epoch_length=512
)

path d:\Machine learning\MI SSL\motor-imagery-classification-2024\models\diffusion
Saving new data
(416, 2, 512)
(416,)
final data shape: (416, 2, 512)
Loading saved data
(208, 2, 512)
(208,)
final data shape: (208, 2, 512)


In [10]:
print(f"path {os.getcwd()}")
files = load_files("../../data/collected_data/")

train_s25_dataset_openbci = OpenBCIDataset(
	subject_splits=TRAIN_SPLIT_OPENBCI,
	dataset=files,
	save_paths=[small_stride],
	fake_data=None,
	dataset_type=OpenBCISubject,
	channels=np.arange(0,2),
	subject_channels=["ch2","ch5"],
	stride=25,
	epoch_length=512
)

test_s25_dataset_openbci = OpenBCIDataset(
	subject_splits=TEST_SPLIT_OPENBCI,
	save_paths=[small_stride],
	fake_data=None,
	dataset_type=OpenBCISubject,
	channels=np.arange(0,2),
	subject_channels=["ch2","ch5"],
	stride=25,
	epoch_length=512
)

path d:\Machine learning\MI SSL\motor-imagery-classification-2024\models\diffusion
Saving new data
(1984, 2, 512)
(1984,)
final data shape: (1984, 2, 512)
Loading saved data
(992, 2, 512)
(992,)
final data shape: (992, 2, 512)


#### CSP

In [11]:

train_csp_dataset_openbci = OpenBCIDataset(
	subject_splits=TRAIN_SPLIT_OPENBCI,
	dataset=files,
	save_paths=[CSP_SAVE_PATH_OPENBCI],
	fake_data=None,
	dataset_type=CSPOpenBCISubject,
	channels=CSP_CHANNELS_OPENBCI,
	subject_channels=["ch2","ch5"],
	stride=128,
	epoch_length=512
)

test_csp_dataset_openbci = OpenBCIDataset(
	subject_splits=TEST_SPLIT_OPENBCI,
	save_paths=[CSP_SAVE_PATH_OPENBCI],
	fake_data=None,
	dataset_type=CSPOpenBCISubject,
	channels=CSP_CHANNELS_OPENBCI,
	subject_channels=["ch2","ch5"],
	stride=128,
	epoch_length=512
)

Saving new data
(416, 18, 512)
(416,)
final data shape: (416, 18, 512)
Loading saved data
(208, 18, 512)
(208,)
final data shape: (208, 18, 512)


---
## CSP

In [12]:
def csp_train_test(train_dset,test_dset):

	x_train,y_train = train_dset.data
	x_test,y_test = test_dset.data

	x_train,y_train = np.float64(x_train),np.float64(y_train)
	x_test,y_test = np.float64(x_test),np.float64(y_test)

	csp = CSP(n_components=x_train.shape[1],reg=None,log=True,norm_trace=False)
	svm = SVC(C=1)

	clf = Pipeline(steps=[("csp",csp),
						("classification",svm)])

	clf.fit(x_train,y_train)

	y_train_pred = clf.predict(x_train)
	y_test_pred = clf.predict(x_test)
	train_acc = accuracy_score(y_train,y_train_pred)
	test_acc = accuracy_score(y_test,y_test_pred)

	print(f"train accuracy: {train_acc}")
	print(f"test accuracy: {test_acc}")
	return test_acc


### BCI Competition

In [13]:
csp_bci_comp_acc = csp_train_test(train_csp_dataset_bci_comp,test_csp_dataset_bci_comp)

Computing rank from data with rank=None


    Using tolerance 9.6 (2.2e-16 eps * 18 dim * 2.4e+15  max singular value)
    Estimated rank (mag): 18
    MAG: rank 18 computed from 18 data channels with 0 projectors
Reducing data rank from 18 -> 18
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 9.6 (2.2e-16 eps * 18 dim * 2.4e+15  max singular value)
    Estimated rank (mag): 18
    MAG: rank 18 computed from 18 data channels with 0 projectors
Reducing data rank from 18 -> 18
Estimating covariance using EMPIRICAL
Done.
train accuracy: 0.7699933906146729
test accuracy: 0.7175368139223561


### OpenBCI

In [14]:
csp_openbci_acc = csp_train_test(train_csp_dataset_openbci,test_csp_dataset_openbci)

Computing rank from data with rank=None
    Using tolerance 2.7 (2.2e-16 eps * 18 dim * 6.6e+14  max singular value)
    Estimated rank (mag): 18
    MAG: rank 18 computed from 18 data channels with 0 projectors
Reducing data rank from 18 -> 18
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None


    Using tolerance 2.6 (2.2e-16 eps * 18 dim * 6.5e+14  max singular value)
    Estimated rank (mag): 18
    MAG: rank 18 computed from 18 data channels with 0 projectors
Reducing data rank from 18 -> 18
Estimating covariance using EMPIRICAL
Done.
train accuracy: 0.7932692307692307
test accuracy: 0.6105769230769231


---
## EEGNet

#### Constants

In [15]:
EEGNET_LR = 1E-3
EEGNET_WEIGHT_DECAY = 1E-4

### BCI Competition

In [16]:
eegnet_bci_comp = base_eegnet.EEGNet(2,224)
eegnet_clf_bci_comp = DeepClassifier(
	model=eegnet_bci_comp,
	save_paths=[REAL_DATA_BCI_COMP],
	train_split=TRAIN_SPLIT_BCI_COMP,
	test_split=TEST_SPLIT_BCI_COMP,
	dataset=None,
	subject_dataset_type=subject_dataset,
	channels=CHANNELS_BCI_COMP,
	length=2.05,
	index_cutoff=512
)

eegnet_bci_comp_acc = eegnet_clf_bci_comp.fit(FABRIC,50,EEGNET_LR,EEGNET_WEIGHT_DECAY)

(3026, 2, 512)
(3026,)
final data shape: (3026, 2, 512)
(2241, 2, 512)
(2241,)
final data shape: (2241, 2, 512)


checkpointing
Epoch [1/50], Training Loss: 0.742, Training Accuracy: 54.23%, Validation Loss: 0.655, Validation Accuracy: 59.11%
checkpointing
Epoch [2/50], Training Loss: 0.648, Training Accuracy: 62.86%, Validation Loss: 0.534, Validation Accuracy: 71.61%
checkpointing
Epoch [3/50], Training Loss: 0.609, Training Accuracy: 65.43%, Validation Loss: 0.505, Validation Accuracy: 75.18%
Min loss: 0.5051339285714286 vs 0.5448660714285715
Epoch [4/50], Training Loss: 0.588, Training Accuracy: 68.21%, Validation Loss: 0.545, Validation Accuracy: 71.70%
Min loss: 0.5051339285714286 vs 0.5224330357142857
Epoch [5/50], Training Loss: 0.573, Training Accuracy: 67.91%, Validation Loss: 0.522, Validation Accuracy: 70.98%
Min loss: 0.5051339285714286 vs 0.5079241071428572
Epoch [6/50], Training Loss: 0.573, Training Accuracy: 69.80%, Validation Loss: 0.508, Validation Accuracy: 71.96%
checkpointing
Epoch [7/50], Training Loss: 0.565, Training Accuracy: 69.33%, Validation Loss: 0.497, Validation Acc

---
### OpenBCI

In [17]:
eegnet_openbci = base_eegnet.EEGNet(2,224)
eegnet_clf_openbci = DeepClassifier(
	model=eegnet_openbci,
	save_paths=[small_stride],
	train_split=train_split,
	test_split=test_split,
	dataset=None,
	dataset_type=OpenBCIDataset,
	subject_dataset_type=OpenBCISubject,
	channels=np.arange(0,2),
	subject_channels=["ch2","ch5"],
	stride=25,
	epoch_length=512,
	index_cutoff=512
	)

eeg_net_openbci_acc = eegnet_clf_openbci.fit(FABRIC,50,EEGNET_LR,EEGNET_WEIGHT_DECAY)

Loading saved data
(1984, 2, 512)
(1984,)
final data shape: (1984, 2, 512)
Loading saved data
(992, 2, 512)
(992,)
final data shape: (992, 2, 512)
checkpointing
Epoch [1/50], Training Loss: 0.760, Training Accuracy: 49.19%, Validation Loss: 0.699, Validation Accuracy: 50.60%
checkpointing
Epoch [2/50], Training Loss: 0.712, Training Accuracy: 54.84%, Validation Loss: 0.693, Validation Accuracy: 54.03%
checkpointing
Epoch [3/50], Training Loss: 0.700, Training Accuracy: 55.04%, Validation Loss: 0.657, Validation Accuracy: 60.48%
Min loss: 0.6572265625 vs 0.687744140625
Epoch [4/50], Training Loss: 0.695, Training Accuracy: 57.76%, Validation Loss: 0.688, Validation Accuracy: 56.45%
Min loss: 0.6572265625 vs 0.678466796875
Epoch [5/50], Training Loss: 0.669, Training Accuracy: 60.23%, Validation Loss: 0.678, Validation Accuracy: 54.84%
checkpointing
Epoch [6/50], Training Loss: 0.651, Training Accuracy: 61.09%, Validation Loss: 0.654, Validation Accuracy: 60.69%
checkpointing
Epoch [7/50

---
## SLC

#### Constants and setup

In [18]:
DIFFUSION_LR = 6E-4
DIFFUSION_N_EPOCHS = 250
DIFFUSION_BATCH_SIZE = 64

TIME_DIM = 12
HIDDEN_CHANNEL = 64
KERNEL_SIZE = 65
NUM_SCALES = 2
DECAY_MIN = 2
DECAY_MAX = 2
ACTIVATION_TYPE = "leaky_relu"

USE_FFT_CONV = KERNEL_SIZE * (2**(NUM_SCALES - 1)) >= 100
NUM_TIMESTEPS = 1024
SCHEDULE = "linear"
START_BETA = 1E-4
END_BETA = 8E-2

In [19]:
def diff_train(fabric,
			   diffusion_model,
			   train_loader):
	optimizer = optim.AdamW(
        diffusion_model.parameters(),
        lr=DIFFUSION_LR,
    )

	diffusion_model,optimizer = fabric.setup(diffusion_model,optimizer)

	train_loader = fabric.setup_dataloaders(train_loader)
	for i in range(DIFFUSION_N_EPOCHS):
			
			epoch_loss = []
			for batch in train_loader:
				
				with fabric.autocast():

					signal,cue = batch
					signal = signal.to(torch.bfloat16)
					cue = cue.to(torch.bfloat16)
					cond = cue.unsqueeze(-1).unsqueeze(-1).repeat(1, 1, SIGNAL_LENGTH).to(DEVICE)
					loss = diffusion_model.train_batch(signal.to(DEVICE),
										 cond=cond)
				loss = torch.mean(loss)
				
				epoch_loss.append(loss.item())
				
				fabric.backward(loss)
				optimizer.step()
				optimizer.zero_grad()
				
			epoch_loss = np.mean(epoch_loss)

			print(f"Epoch {i} loss: {epoch_loss}")

fabric_train = lambda model,loader : diff_train(FABRIC,model,loader)

In [30]:
def diffusion_loop(
		diffusion_dset,
		dset_path,
		train_split,
		test_split,
		save_folder,
		name,
		clf_head,
		train=True,
		**head_kwargs):
	
	if not os.path.isdir(save_folder):
		os.makedirs(save_folder)

	network = LongConv(
			signal_length=SIGNAL_LENGTH,
			signal_channel=2, # The CSP classifier components
			time_dim=TIME_DIM,
			cond_channel=network_yaml["cond_channel"], # The cond channel will contain the cue (0 or 1)
			hidden_channel=HIDDEN_CHANNEL,
			in_kernel_size=KERNEL_SIZE,
			out_kernel_size=KERNEL_SIZE,
			slconv_kernel_size=KERNEL_SIZE,
			num_scales=NUM_SCALES,
			decay_min=DECAY_MIN,
			decay_max=DECAY_MAX,
			heads=network_yaml["heads"],
			activation_type=ACTIVATION_TYPE,
			use_fft_conv=USE_FFT_CONV,
		)
	
	noise_sampler = WhiteNoiseProcess(1.0, SIGNAL_LENGTH)

	diffusion_model = Diffusion(
		network=network,
		diffusion_time_steps=NUM_TIMESTEPS,
		noise_sampler=noise_sampler,
		mal_dist_computer=noise_sampler,
		schedule=SCHEDULE,
		start_beta=START_BETA,
		end_beta=END_BETA,
	)

	train_loader = DataLoader(
		diffusion_dset,
		DIFFUSION_BATCH_SIZE,
	)

	fabric_train = lambda fabric : diff_train(fabric,diffusion_model,train_loader)
	if train:
		FABRIC.launch(fabric_train)

	diff_path = os.path.join(save_folder,name)

	torch.save(diffusion_model.state_dict(),diff_path)

	diffusion_model.load_state_dict(torch.load(diff_path))

	head = clf_head(**head_kwargs)

	clf = DiffusionClf(diffusion_model,head,freeze=False)

	clf.to(DEVICE)

	slc_clf = DeepClassifier(
		model=clf.to(DEVICE),
		save_paths=[dset_path],
		fake_data=None,
		train_split=train_split,
		test_split=test_split,
		**head_kwargs
	)

	optimizer = optim.AdamW([
		{"params":slc_clf.model.model.parameters(),"lr":2E-5,"weight_decay":1E-4},
		{"params":slc_clf.model.clf.parameters(),"lr":1E-3,"weight_decay":1E-4}
	])

	test_acc = slc_clf.fit(fabric=FABRIC,
			 num_epochs=25,
			 lr=1E-3,
			 weight_decay=1E-4,
			 verbose=True,
			 optimizer=optimizer)
	
	print(f"\n\nTest acc: {test_acc}")

	return test_acc

### BCI Competition

In [31]:
diff_bci_comp_acc = diffusion_loop(
	diffusion_dset=train_dataset_bci_comp,
	dset_path=REAL_DATA_BCI_COMP,
	train_split=TRAIN_SPLIT_BCI_COMP,
	test_split=TEST_SPLIT_BCI_COMP,
	save_folder="results/saved_models/bci_comp",
	name="slc_eegnet.pt",
	clf_head=EEGNetHead,
	c_in=128,
	d_out=256,
	dataset=None,
	subject_dataset_type=subject_dataset,
	length=2.05,
	index_cutoff=512,
	channels=CHANNELS_BCI_COMP,
	train=True
)

(3026, 2, 512)
(3026,)
final data shape: (3026, 2, 512)
(2241, 2, 512)
(2241,)
final data shape: (2241, 2, 512)
using specified optimizer
checkpointing
Epoch [1/25], Training Loss: 0.758, Training Accuracy: 50.26%, Validation Loss: 0.715, Validation Accuracy: 51.34%
checkpointing
Epoch [2/25], Training Loss: 0.698, Training Accuracy: 53.93%, Validation Loss: 0.682, Validation Accuracy: 56.52%
checkpointing
Epoch [3/25], Training Loss: 0.671, Training Accuracy: 59.45%, Validation Loss: 0.629, Validation Accuracy: 63.66%
checkpointing
Epoch [4/25], Training Loss: 0.640, Training Accuracy: 63.19%, Validation Loss: 0.580, Validation Accuracy: 66.52%
checkpointing
Epoch [5/25], Training Loss: 0.586, Training Accuracy: 68.08%, Validation Loss: 0.541, Validation Accuracy: 70.89%
checkpointing
Epoch [6/25], Training Loss: 0.564, Training Accuracy: 69.30%, Validation Loss: 0.536, Validation Accuracy: 71.16%
Min loss: 0.5356584821428572 vs 0.5575334821428571
Epoch [7/25], Training Loss: 0.527, T

### OpenBCI

In [27]:
diff_openbci_acc = diffusion_loop(
	diffusion_dset=train_s25_dataset_openbci,
	dset_path=SAVE_PATH_OPENBCI,
	train_split=TRAIN_SPLIT_OPENBCI,
	test_split=TEST_SPLIT_OPENBCI,
	save_folder="results/saved_models/openbci",
	name="slc_eegnet.pt",
	clf_head=EEGNetHead,
	c_in=128,
	d_out=256,
	dataset=None,
	dataset_type = OpenBCIDataset,
	subject_dataset_type=OpenBCISubject,
	subject_channels=["ch2","ch5"],
	length=2.0,
	epoch_length=512,
	index_cutoff=512,
	channels=CHANNELS_OPENBCI,
	train=True
)

Loading saved data
(416, 2, 512)
(416,)
final data shape: (416, 2, 512)
Loading saved data
(208, 2, 512)
(208,)
final data shape: (208, 2, 512)
using specified optimizer
checkpointing
Epoch [1/25], Training Loss: 0.746, Training Accuracy: 55.05%, Validation Loss: 0.696, Validation Accuracy: 50.96%
checkpointing
Epoch [2/25], Training Loss: 0.734, Training Accuracy: 50.24%, Validation Loss: 0.691, Validation Accuracy: 45.19%
Min loss: 0.69140625 vs 0.7060546875
Epoch [3/25], Training Loss: 0.702, Training Accuracy: 53.12%, Validation Loss: 0.706, Validation Accuracy: 47.12%
Min loss: 0.69140625 vs 0.7216796875
Epoch [4/25], Training Loss: 0.679, Training Accuracy: 56.01%, Validation Loss: 0.722, Validation Accuracy: 49.04%
Min loss: 0.69140625 vs 0.708984375
Epoch [5/25], Training Loss: 0.684, Training Accuracy: 57.69%, Validation Loss: 0.709, Validation Accuracy: 52.88%
Min loss: 0.69140625 vs 0.7294921875
Epoch [6/25], Training Loss: 0.660, Training Accuracy: 61.54%, Validation Loss: 

---

## Results

In [23]:
print(f"""

BCI Competition
--------
CSP: {csp_bci_comp_acc}
EEGNet: {eegnet_bci_comp_acc}
Pre-trained: {diff_bci_comp_acc}

OpenBCI
--------
CSP: {csp_openbci_acc}
EEGNet: {eeg_net_openbci_acc}
Pre-trained: {diff_openbci_acc}

""")



BCI Competition
--------
CSP: 0.7175368139223561
EEGNet: 78.48214285714286
Pre-trained: 77.41071428571429

OpenBCI
--------
CSP: 0.6105769230769231
EEGNet: 64.91935483870968
Pre-trained: 59.61538461538461


